<h1 style="text-align: center;">Schedule Optimization Effort</h1>

## Necessary Libraries

In [2]:
import pandas as pd

## Loading in the Data

In [3]:
big_west_runs = pd.read_csv('files/big_west_runs.csv').copy()
big_west_runs

,Team,R/GP,RA/GP,R/IP,RA/IP,ATT,DEF
0,Cal Baptist,6.270833,6.208333,0.690367,0.683486,1.000794,0.994159
1,Cal Poly,7.396226,5.169811,0.808247,0.564948,1.171680,0.821741
2,Cal State Bakersfield,4.367347,6.224490,0.476615,0.679287,0.690927,0.988052
3,Cal State Fullerton,6.120000,5.940000,0.665217,0.645652,0.964335,0.939128
4,Cal State Northridge,6.200000,7.960000,0.672451,0.863341,0.974822,1.255765
5,Long Beach State,5.557692,6.615385,0.613588,0.730361,0.889491,1.062341
6,Sacramento State,5.200000,5.040000,0.593607,0.575342,0.860526,0.836860
7,UC Irvine,8.777778,5.444444,0.967347,0.600000,1.402319,0.872725
8,UC Riverside,4.253968,8.460317,0.471002,0.936731,0.682790,1.362515
9,UC San Diego,7.400000,6.120000,0.813187,0.672527,1.178840,0.978219
